In [1]:
  import pandas as pd
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt
import scipy
with open("/content/drive/MyDrive/Colab Notebooks/train.npy", "rb") as f:
    data = np.load(f)
    images = data["images"]
    labels = data["labels"]
    f.close()
  
X_data = images / 255 - 0.5
y_data = np.zeros((len(labels), 10))
y_data[np.arange(len(labels)), labels] = 1.0

# Shuffle the indices of our dataset
rng = np.random.RandomState(5)
indices = np.arange(len(X_data))
rng.shuffle(indices)

# Split the indices into two parts
test_fraction = 0.001
test_count = int(len(indices) * test_fraction)
train_indices = indices[test_count:]
test_indices = indices[:test_count]

# Create the two datasets (training and testing) using our shuffled indices
X_train = X_data[train_indices]
y_train = y_data[train_indices]
X_test = X_data[test_indices]
y_test = y_data[test_indices]
print(y_test.shape)
X_orig = images[train_indices]

(73, 10)


In [ ]:
from keras.layers import LeakyReLU
def residual_layers(x, neuron_count=256):
    skipping_input = x

    x = tf.keras.layers.Conv2D(neuron_count // 4, kernel_size=(1, 1), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same")(x)
    x = tf.keras.layers.Conv2D(neuron_count // 4, kernel_size=(3, 3), activation=LeakyReLU(alpha=0.1), padding="same")(x)
    x = tf.keras.layers.Conv2D(neuron_count, kernel_size=(1, 1), padding="same")(x)

    x = skipping_input + x
    x = tf.keras.layers.Activation(LeakyReLU(alpha=0.1))(x)

    return x

inputs = tf.keras.layers.Input(shape=(32, 32, 3))
x = tf.keras.layers.Conv2D(64, kernel_size=(7, 7), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same")(inputs)


x = residual_layers(x, neuron_count=64)
x = residual_layers(x, neuron_count=64)
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(0.5)
x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(x)

x = tf.keras.layers.Conv2D(128, kernel_size=(3, 3), strides=(2, 2), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same")(x)
#x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(x) Neverta antro poolingo
x = residual_layers(x, neuron_count=128)
x = residual_layers(x, neuron_count=128)
tf.keras.layers.BatchNormalization() # BN VEIKIA GERIAU PRIES DROPOUT
tf.keras.layers.Dropout(0.5)

x = tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation=tf.keras.layers.LeakyReLU(alpha=0.1), padding="same")(x)
x = residual_layers(x, neuron_count=256)
x = residual_layers(x, neuron_count=256)
tf.keras.layers.BatchNormalization()
tf.keras.layers.Dropout(0.5)

#x = tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(x) Trecio irgi

#x = tf.keras.layers.GlobalAveragePooling2D()(x) Global neapsimoka
x = tf.keras.layers.Flatten()(x)


outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
model.summary()

class CIFAR10Sequence(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

        self.rng = np.random.RandomState(0)
        self.indices = np.arange(len(self.x))

        self.on_epoch_end()

        self.transform_rng = np.random.RandomState(0)

    def __len__(self):
        return int(np.ceil(len(self.x) / self.batch_size))

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = self.x[batch_indices]
        batch_y = self.y[batch_indices]

        for i in range(len(batch_x)):
            angle = self.transform_rng.uniform(-25, 25)
            batch_x[i] = scipy.ndimage.rotate(batch_x[i], angle, reshape=False, mode="reflect")

            shift_x = self.transform_rng.randint(-2, 3)
            shift_y = self.transform_rng.randint(-2, 3)

            batch_x[i] = scipy.ndimage.shift(batch_x[i], (shift_x, shift_y, 0), mode="reflect")

        return batch_x / 255 - 0.5, batch_y#  / 255 - 0.5

    def on_epoch_end(self):
        self.rng.shuffle(self.indices)

generator = CIFAR10Sequence(X_orig, y_train, batch_size=256)

  
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

def step_decay(epoch):
    lr = 0.0006 * 0.94 ** epoch
    print(f"Current learning rate: {lr} in epoch: {epoch}")
    return lr

model.fit(generator, batch_size=256, epochs=30) # kol kas be step decay

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   1040        conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 32, 16)   2320        conv2d_1[0][0]                   
______________________________________________________________________________________________

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=0)

print(f"Train accuracy = {accuracy:.4f}")

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)

print(f"Test accuracy = {accuracy:.4f}")

import numpy as np
with open("/content/drive/MyDrive/Colab Notebooks/test.npy", "rb") as f:
    duom = np.load(f)
    XX_test = duom["images"]
    f.close()

XX_test = XX_test / 255 - 0.5
Prediction = model.predict(XX_test, verbose=2)
Prediction = tf.argmax(Prediction, axis=1)
print(Prediction)
Prediction = pd.DataFrame(Prediction).to_csv('/content/drive/MyDrive/Colab Notebooks/PredictionLeakySUADAM.csv', header=None, index=None)